In [26]:
import json
import re
import pandas as pd

train 데이터셋

In [3]:
file_path = '../data/movie/train.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [2]:
# 불용어 목록 정의
stopwords = {"a", "an", "the", "and", "or", "of", "to", "in", "on", "with", "by", "for", "at", "from", "is"}

# 영화 제목 전처리 함수 정의
def normalize_title(title):
    if not isinstance(title, str):
        return ''
    title = re.sub(r'\(.*?\)', '', title)  # 괄호 안 내용 제거
    title = re.sub(r'[^\w\s]', '', title)  # 특수문자 제거
    title = title.lower().strip()  # 소문자로 변환 및 공백 제거
    # 불용어 제거
    title_words = title.split()
    filtered_words = [word for word in title_words if word not in stopwords]
    return ' '.join(filtered_words)

# 영화 제목 추출 함수 정의
def extract(input_text):
    return re.findall(r'"(.*?)"', input_text)

In [4]:
inputs = [extract(entry['input']) for entry in data]
inputs_normalized = [[normalize_title(item) for item in sublist] for sublist in inputs]

The Movies Dataset (kaggle)

In [6]:
df = pd.read_csv('../data/movie_overview/movies_metadata.csv')
df.head(1)

/tmp/ipykernel_16866/4277639516.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/movie_overview/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [7]:
df['title_normalized'] = df['title'].apply(normalize_title)
overview_dict = dict(zip(df['title_normalized'], df['overview']))

In [8]:
# 최종 결과 저장을 위한 리스트 초기화
result = []

# inputs의 각 리스트 요소들에 대해 overview 찾기
for original_list, normalized_list in zip(inputs, inputs_normalized):
    for original_title, normalized_title in zip(original_list, normalized_list):
        overview = overview_dict.get(normalized_title, None)
        result.append({'original_title': original_title, 'overview': overview})

# 데이터프레임으로 변환 및 CSV 파일로 저장
final_df = pd.DataFrame(result)
final_df.to_csv('../data/movie_overview/movie_overview_result.csv', index=False)

print("작업 완료! 데이터프레임이 성공적으로 저장되었습니다.")

작업 완료! 데이터프레임이 성공적으로 저장되었습니다.


MovieLens (GroupLens)

In [56]:
df2 = pd.read_csv('./data/new/movies.csv')
df2['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Orca (2023)', 'The Angry Breed (1968)',
       'Race to the Summit (2023)'], dtype=object)

In [57]:
no_exist_2=[]

for i in movie_list:
    if i not in df2['title'].unique():
        no_exist_2.append(i)
        
print(no_exist_2)
print(len(no_exist_2))

['Cinema Paradiso (1988)', 'Bride of Frankenstein (1935)', 'Willy Wonka and the Chocolate Factory (1971)', 'Pink Floyd - The Wall (1982)', 'Up in Smoke (1978)', 'Aladdin and the King of Thieves (1996)', 'Koyaanisqatsi (1983)', 'As Good As It Gets (1997)', "Devil's Advocate, The (1997)", 'Boot, Das (1981)', 'Star Wars (1977)', 'Return of the Jedi (1983)', "Things to Do in Denver when You're Dead (1995)", 'Twelve Monkeys (1995)', 'Star Trek: The Wrath of Khan (1982)', 'Monty Python and the Holy Grail (1974)', 'Dolores Claiborne (1994)', 'Raiders of the Lost Ark (1981)', 'Good, The Bad and The Ugly, The (1966)', 'Three Colors: White (1994)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 'Magnificent Seven, The (1954)', 'M*A*S*H (1970)', 'GoodFellas (1990)', 'Shall We Dance? (1937)', "William Shakespeare's Romeo and Juliet (1996)", 'Sunset Blvd. (1950)', 'Seven (Se7en) (1995)', 'Three Colors: Red (1994)', 'Nikita (La Femme Nikita) (1990)', 'Three Colors: Bl

MovieLens 100K Dataset (kaggle)

In [75]:
u_cols =  ['movie id', 'movie title', 'release date', 'video release date',
'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
'Thriller', 'War', 'Western'
]
df3 = pd.read_csv('./data/new/u.item', sep='|', names=u_cols,
encoding='latin-1').drop('unknown', axis='columns')
df3.head(2)

,movie id,movie title,release date,video release date,IMDb URL,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [65]:
no_exist_3=[]

for i in movie_list:
    if i not in df3['2'].unique():
        no_exist_3.append(i)
        
print(no_exist_3)
print(len(no_exist_3))

[]
0


genome-scores

In [7]:
df4 = pd.read_csv('../data/movie_10m/movies.csv')
df4['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'The Pirates (2014)', 'Rentun Ruusu (2001)',
       'Innocence (2014)'], dtype=object)

In [8]:
df4['title']

0                          Toy Story (1995)
1                            Jumanji (1995)
2                   Grumpier Old Men (1995)
3                  Waiting to Exhale (1995)
4        Father of the Bride Part II (1995)
                        ...                
27273          Kein Bund für's Leben (2007)
27274         Feuer, Eis & Dosenbier (2002)
27275                    The Pirates (2014)
27276                   Rentun Ruusu (2001)
27277                      Innocence (2014)
Name: title, Length: 27278, dtype: object

In [9]:
no_exist_4=[]

for i in movie_list:
    if not any (i in title for title in df4['title'].unique()):
        no_exist_4.append(i)
        
print(no_exist_4)
print(len(no_exist_4))

['Cinema Paradiso (1988)', 'Bride of Frankenstein (1935)', 'Willy Wonka and the Chocolate Factory (1971)', 'Pink Floyd - The Wall (1982)', 'Koyaanisqatsi (1983)', 'As Good As It Gets (1997)', 'Boot, Das (1981)', 'Star Wars (1977)', "Things to Do in Denver when You're Dead (1995)", 'Twelve Monkeys (1995)', 'Star Trek: The Wrath of Khan (1982)', 'Monty Python and the Holy Grail (1974)', 'Dolores Claiborne (1994)', 'Raiders of the Lost Ark (1981)', 'Good, The Bad and The Ugly, The (1966)', 'Three Colors: White (1994)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)', 'Magnificent Seven, The (1954)', 'M*A*S*H (1970)', 'GoodFellas (1990)', 'Shall We Dance? (1937)', "William Shakespeare's Romeo and Juliet (1996)", 'Sunset Blvd. (1950)', 'Seven (Se7en) (1995)', 'Three Colors: Red (1994)', 'Nikita (La Femme Nikita) (1990)', 'Three Colors: Blue (1993)', 'Professional, The (1994)', "What's Love Got to Do with It (1993)", 'Conan the Barbarian (1981)', 'Transformers: 

In [14]:
import re

def normalize_text(text):
    # 소문자로 변환
    text = text.lower()
    # '&', 'and', '+'를 모두 'and'로 통일
    text = re.sub(r"\b(and|&|\+)\b", "and", text)
    # 연도 형식 (예: '(1995)') 제거
    text = re.sub(r"\(\d{4}\)", "", text)
    # 특수문자를 공백으로 변환 (예: '-', ':', ',', '(', ')')
    text = re.sub(r"[-:(),]", " ", text)
    # 여러 개의 공백을 하나로 압축
    text = re.sub(r"\s+", " ", text)
    return text.strip()

no_exist_4 = []

# movie_list를 정규화하여 전처리
normalized_movie_list = [normalize_text(movie) for movie in movie_list]

# df4['title']의 각 항목을 정규화하여 준비
normalized_titles = [normalize_text(title) for title in df4['title'].unique()]

# movie_list의 각 영화 제목을 df4의 제목과 비교
for movie, normalized_movie in zip(movie_list, normalized_movie_list):
    words = normalized_movie.split()
    
    found = False
    # df4의 제목에서 모든 단어가 포함되는지 확인
    for normalized_title in normalized_titles:
        if all(word in normalized_title for word in words):
            found = True
            break
    
    # 모든 단어가 포함된 제목이 없으면 no_exist_4에 추가
    if not found:
        no_exist_4.append(movie)

print(no_exist_4)
print(len(no_exist_4))


['Willy Wonka and the Chocolate Factory (1971)', "William Shakespeare's Romeo and Juliet (1996)", "Jackie Chan's First Strike (1996)", 'Ghost in the Shell (Kokaku kidotai) (1995)', 'U.S. Marshalls (1998)', 'Magic Hour, The (1998)', "Mat' i syn (1997)", "Stephen King's The Langoliers (1995)", 'When the Cats Away (Chacun cherche son chat) (1996)', 'Withnail and I (1987)', 'Duoluo tianshi (1995)', "Robert A. Heinlein's The Puppet Masters (1994)", 'Celestial Clockwork (1994)', 'Bitter Sugar (Azucar Amargo) (1996)']
14


movie_genre 학습 json 파일

In [48]:
df = pd.read_csv('../data/movie_genre/title_genre.csv')
df

,title,genre
0,Toy Story (1995),"animation, children's, comedy"
1,GoldenEye (1995),"action, adventure, thriller"
2,Four Rooms (1995),thriller
3,Get Shorty (1995),"action, comedy, drama"
4,Copycat (1995),"crime, drama, thriller"
...,...,...
1657,Sweet Nothing (1995),drama
1658,Mat' i syn (1997),drama
1659,B. Monkey (1998),"romance, thriller"
1660,You So Crazy (1994),comedy


In [49]:
genre_dict = df.set_index('title')['genre'].to_dict()
genre_dict


{'Toy Story (1995)': "animation, children's, comedy",
 'GoldenEye (1995)': 'action, adventure, thriller',
 'Four Rooms (1995)': 'thriller',
 'Get Shorty (1995)': 'action, comedy, drama',
 'Copycat (1995)': 'crime, drama, thriller',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 'drama',
 'Twelve Monkeys (1995)': 'drama, sci-fi',
 'Babe (1995)': "children's, comedy, drama",
 'Dead Man Walking (1995)': 'drama',
 'Richard III (1995)': 'drama, war',
 'Seven (Se7en) (1995)': 'crime, thriller',
 'Usual Suspects, The (1995)': 'crime, thriller',
 'Mighty Aphrodite (1995)': 'comedy',
 'Postino, Il (1994)': 'drama, romance',
 "Mr. Holland's Opus (1995)": 'drama',
 'French Twist (Gazon maudit) (1995)': 'comedy, romance',
 'From Dusk Till Dawn (1996)': 'action, comedy, crime, horror, thriller',
 'White Balloon, The (1995)': 'drama',
 "Antonia's Line (1995)": 'drama',
 'Angels and Insects (1995)': 'drama, romance',
 'Muppet Treasure Island (1996)': 'action, adventure, comedy, musical, thr

In [58]:
import json
with open ('../data/movie/valid.json', 'r') as file:
    data = json.load(file)
    
data[0]

{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".',
 'input': 'User Preference: "Silence of the Lambs, The (1991)", "Indiana Jones and the Last Crusade (1989)", "Taxi Driver (1976)", "My Favorite Year (1982)", "Titanic (1997)"\nUser Unpreference: "Absolute Power (1997)", "Escape from L.A. (1996)", "Batman & Robin (1997)", "English Patient, The (1996)", "Tomorrow Never Dies (1997)"\nWhether the user will like the target movie "Scream (1996)"?',
 'output': 'No.'}

In [59]:
genre_data = data

for entry in genre_data:
    entry['instruction'] = 'Answer whether the user will like the target movie, considering the user\'s preferred and non-preferred movies. The movie is presented in the form of "the title (year) [genre]", and the answer is only "Yes." or "No.".' 
    
genre_data[0]

{'instruction': 'Answer whether the user will like the target movie, considering the user\'s preferred and non-preferred movies. The movie is presented in the form of "the title (year) [genre]", and the answer is only "Yes." or "No.".',
 'input': 'User Preference: "Silence of the Lambs, The (1991)", "Indiana Jones and the Last Crusade (1989)", "Taxi Driver (1976)", "My Favorite Year (1982)", "Titanic (1997)"\nUser Unpreference: "Absolute Power (1997)", "Escape from L.A. (1996)", "Batman & Robin (1997)", "English Patient, The (1996)", "Tomorrow Never Dies (1997)"\nWhether the user will like the target movie "Scream (1996)"?',
 'output': 'No.'}

In [60]:
import re

def update_input(data_list, genre_dict):
    for entry in data_list:
        movies = re.findall(r'"(.*?)"', entry['input'])
        updated_movies = []
    
        for movie in movies:
            genre = genre_dict.get(movie, "[genre unknown]")
            updated_movie = f'{movie} [{genre}]'
            updated_movies.append(f'"{updated_movie}"')
    
        updated_input = entry['input']
        for original, updated in zip(movies, updated_movies):
            updated_input = updated_input.replace(f'"{original}"', updated, 1)
        
        entry['input'] = updated_input
        
    return data_list

genre_data = update_input(genre_data, genre_dict)
genre_data

[{'instruction': 'Answer whether the user will like the target movie, considering the user\'s preferred and non-preferred movies. The movie is presented in the form of "the title (year) [genre]", and the answer is only "Yes." or "No.".',
  'input': 'User Preference: "Silence of the Lambs, The (1991) [drama, thriller]", "Indiana Jones and the Last Crusade (1989) [action, adventure]", "Taxi Driver (1976) [drama, thriller]", "My Favorite Year (1982) [comedy]", "Titanic (1997) [action, drama, romance]"\nUser Unpreference: "Absolute Power (1997) [mystery, thriller]", "Escape from L.A. (1996) [action, adventure, sci-fi, thriller]", "Batman & Robin (1997) [action, adventure, crime]", "English Patient, The (1996) [drama, romance, war]", "Tomorrow Never Dies (1997) [action, romance, thriller]"\nWhether the user will like the target movie "Scream (1996) [horror, thriller]"?',
  'output': 'No.'},
 {'instruction': 'Answer whether the user will like the target movie, considering the user\'s preferr

In [61]:
with open ('../data/movie_genre/valid.json', 'w', encoding='utf-8') as file:
    json.dump(genre_data, file, indent=4)
    
print("done")

done
